# Team Formation Problem

Assume that you have a set of players where each player, $i$, has a skill level, $s_{i}$.  We
would like to assign the players to $m$ teams of size 4 such that the teams are "balanced" (all
teams have approximately the same total skill level).  We define a team's skill level as the sum of the team members' skill levels.

So, our objective is to minimize the maximum deviation from "average" over all teams:
\begin{align}
\text{Min} \ \text{Max}_{j} \quad \left| \sum_{i} s_{i}x_{ij} - \frac{\sum_{i} s_{i}}{n}\right|
\end{align}

After adjusting to remove the absolute values, the formulation is:

\begin{align*}
\text{Min} \quad & z \\
\text{s.t.} \quad & z \ge \sum_{i} s_{i} x_{ij} - \frac{\sum_{i}{s_i}}{n} \quad \forall ~ j \\
& z\ge \frac{\sum_{i}{s_i}}{n} - \sum_{i} s_{i} x_{ij} \quad \forall ~ j \\
& \sum_{j} x_{ij} \le 1 \quad \forall ~ i \\
& \sum_{i} x_{ij} = 4 \quad \forall ~ j \\
& z \ge 0 \\
& x_{ij} \in \{0, 1\}
\end{align*}



In [2]:
# --------------------------
# equalize_teams.py - forms teams by eqalizing the total team skill.
#
# 2015-09-06 - jeff smith with optimization help from chase murray
# --------------------------
from gurobipy import *

# set to 1 if you want to see the gurobi intermediate results
show_intermediate = 1

# Read in the players from an external file.  The format of the file
# is 'player, skill' for each line. Skill should be interger.
players = []
infile = open('players.txt', 'r')
pline = infile.readline().rstrip()
itr = 0
while (pline):
    try:
        player = pline.split(',')
        name = player[0]
        skill = int(player[1])
    except Exception, e:
        print 'Exception on {}: [{}]'.format(player, e)
    else:
        players.append([])
        players[itr].append(name)
        players[itr].append(skill)        
        itr += 1
    pline = infile.readline().rstrip()

# total number of players
num_players = len(players)
# team size (players/team)
team_size = 4
# number of teams
num_teams = num_players/team_size
# total skill level over all players
total_skill = sum([p[1] for p in players])
# average team skill
avg_team_skill = float(total_skill)/num_teams
print "{} players; {} teams of size {} with {} leftovers.".format(
    len(players) 
    ,num_teams 
    ,team_size
    ,num_players - num_teams*team_size
    )
print "Total skill {}; avg team skill {:.2f}\n".format(total_skill, avg_team_skill)

# Define the model
m = Model("Team Formation")
# Limit to 2 minutes
m.setParam("TimeLimit", 60)
# turn off intermediate output
if show_intermediate == 0:
    m.setParam('OutputFlag',False)

# decvarx - x[i][j] = 1 if person i is assigned to team j
decvarx = []
for i in range(num_players):
    decvarx.append([])
    for j in range(num_teams):
        decvarx[i].append(m.addVar(obj=0,vtype=GRB.BINARY))

# z used to specify balance (deviation of a team's skill from the average team skill)
z = m.addVar(lb=0, ub=total_skill, obj=1, vtype=GRB.CONTINUOUS)

# The objective is to minimize the total cost.
m.modelSense = GRB.MINIMIZE

# Update model to integrate new variables
m.update()

# Each person is assigned to exactly 1 team
for i in range(num_players):
    m.addConstr(quicksum(decvarx[i][j] for j in range(num_teams)) <= 1)

for j in range(num_teams):
    # each team has the right number of members
    m.addConstr(quicksum(decvarx[i][j] for i in range(num_players)) == team_size)
    # defining z based on teams' deviations from the mean
    m.addConstr(z >= quicksum(players[i][1]*decvarx[i][j] for i in range(num_players)) - avg_team_skill)
    m.addConstr(z >= avg_team_skill - quicksum(players[i][1]*decvarx[i][j] for i in range(num_players)))

m.optimize()
if m.Status == GRB.OPTIMAL:
    print '\nGurobi reports the solution is optimal'
else:
    print '\nSolution might not be optimal (Gurobi optimal flag not set)'
# Interpret the solution - create the teams from the decision variables
teams = []
for j in range(num_teams):
    teams.append([])
for i in range(num_players):
    for j in range(num_teams):
        if decvarx[i][j].x == 1:
            teams[j].append(i)
            players[i].append(j)
# Print the problem details            
print "\n{} players; {} teams of size {} with {} leftovers.".format(
    len(players) 
    ,num_teams 
    ,team_size
    ,num_players - num_teams*team_size
    )
print "Total skill {}; avg team skill {:.2f}".format(total_skill, avg_team_skill)
# Print the solution
print "\nSolution - Objective function value: {:.4f}; Teams:".format(m.objVal)
i = 1
for t in teams:
    # accumulate the team skill level
    team_skill = 0
    # build the team member string to print later
    team_members = ""
    for j in range(team_size):
        # increment team_skill from the players list
        team_skill += players[t[j]][1]
        # add the player name from the players list
        team_members += "{} ({}), ".format (players[t[j]][0], players[t[j]][1])
    # the .replace gets rid of the last comma in the team_members strig
    print "[{:2d}] ({:.1f}): {}".format(i, team_skill, team_members.replace(',','')[:-1])
    i += 1

# print the rejects (if there are any)
if num_players - num_teams*team_size > 0:
    print "\nLeftover players:"
    for p in players:
        if len(p) < 3:
            print '\t{} ({})'.format(p[0], p[1])    


Exception on ['Mendy Dimery', '']: [invalid literal for int() with base 10: '']
47 players; 11 teams of size 4 with 3 leftovers.
Total skill 516; avg team skill 46.91

Changed value of parameter TimeLimit to 60.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 80 rows, 518 columns and 2068 nonzeros
Variable types: 1 continuous, 517 integer (517 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+02]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 12.0909
Presolve time: 0.01s
Presolved: 80 rows, 518 columns, 2068 nonzeros
Variable types: 1 continuous, 517 integer (517 binary)

Root relaxation: objective 7.272727e-01, 228 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.72727    0   32   12.09091    0.72727  94.0%   